# NLU+ 2021-2022 Lab 3: Tensor Computation in PyTorch

#### Authors: Yao Fu and Frank Keller

This lab indends to teach computation with tensors, which is a fundamental paradigm in modern machine learning. 

Students should work through Section 1 before the lab session, as this section introduces the basics of tensor computation. The lab session will then focus on more advanced computations with tensors, in Sections 2 and 3.

**Students are strongly encouraged to complete this lab before starting CW2. Many computations in the lab will be encountered in CW2 again, so the amount of difficult in the coursework will be significantly reduced.**

We suggest using jupyter lab instead of jupyter notebook. Basically jupyter lab is an enhanced version of jupyter notebook. Downloading jupyter lab is simply:

```bash
pip install jupyterlab
```

Then in a terminal, start jupyter lab with 
```bash
jupyter lab
```

In [1]:
import torch 
import torch.nn.functional as F

# Section 1. Basic Tensor Computation

## Definition of Tensor: High-dimensional Array

### Vector

A vector is a rank-1 (or one-dimentional) tensor.

In NLP, it could be a single sentence where each word is an integer index.

In [2]:
sent = torch.tensor([1, 3, 5, 7, 9])

In [3]:
sent

tensor([1, 3, 5, 7, 9])

### Matrix

A matrix is a rank-2 (or two-dimentional) tensor.

In NLP, this could used to represent a batch of different sentences.

In [4]:
sents = torch.tensor([[1, 3, 5, 7, 9], 
                     [2, 4, 6, 8, 10]]) # a batch of two sentences

In [5]:
sents # we assume these index corresponds to some actual words

tensor([[ 1,  3,  5,  7,  9],
        [ 2,  4,  6,  8, 10]])

### Rank-3 Tensor

There can be a rank-3 (three-dimensional) tensor.

In NLP, this could be a batch of sentences, where each word in each sentence is representened as an embedding vector.

In [6]:
sents_emb = torch.rand([2, 5, 10])  # [batch_size = 2, sentence_length = 5, hidden_size = 10]
                                    # we use random vectors for the purpose of demonstration 
                                    # we use the name `hidden_size` because the embedding is usually referred as 'hidden representations' in neural networks

In [7]:
sents_emb

tensor([[[0.5888, 0.1384, 0.3219, 0.5481, 0.8978, 0.3362, 0.5466, 0.3833,
          0.6710, 0.8290],
         [0.1809, 0.0470, 0.9739, 0.8530, 0.5467, 0.4967, 0.0050, 0.4975,
          0.1456, 0.0031],
         [0.1117, 0.2034, 0.1183, 0.9997, 0.9059, 0.7601, 0.3395, 0.5432,
          0.7731, 0.3103],
         [0.7531, 0.3300, 0.7945, 0.8864, 0.4057, 0.4517, 0.6010, 0.8064,
          0.3448, 0.6773],
         [0.3902, 0.9508, 0.0760, 0.8834, 0.9523, 0.4662, 0.0296, 0.1755,
          0.9334, 0.5587]],

        [[0.1730, 0.7177, 0.2852, 0.0100, 0.7166, 0.9565, 0.7168, 0.0939,
          0.6218, 0.2003],
         [0.4821, 0.5393, 0.0591, 0.4795, 0.4916, 0.5234, 0.5107, 0.3697,
          0.6168, 0.5117],
         [0.4011, 0.3225, 0.3118, 0.0167, 0.5144, 0.0053, 0.8730, 0.0512,
          0.1928, 0.0469],
         [0.4480, 0.2642, 0.2911, 0.6106, 0.7011, 0.2939, 0.1632, 0.4831,
          0.9947, 0.6860],
         [0.8765, 0.6472, 0.8652, 0.9155, 0.8116, 0.7634, 0.0687, 0.8537,
          0.204

Here `sents_emb[i, j]` means the word embedding for the j-th word in the i-th sentence.

In [8]:
#  word embedding for 0-th sentence, 3-rd word
sents_emb[0, 3]

tensor([0.7531, 0.3300, 0.7945, 0.8864, 0.4057, 0.4517, 0.6010, 0.8064, 0.3448,
        0.6773])

### Rank-4 Tensor

As for a rank-4 (four-dimensional) tensor, this could be a batch of sentences, where each word of each sentence is further devided into different heads of attention keys for multi-head attention (you will encounter this in CW2).

In [17]:
sents_emb_keys = torch.rand([2, 5, 4, 10])  # [batch_size = 2, sentence_length = 5, number_of_heads = 4, hidden_size = 10]
                                            # Again, we use random vectors for the purpose of demonstration
sents_emb_keys

tensor([[[[0.8334, 0.8729, 0.9025, 0.6452, 0.3510, 0.8613, 0.7899, 0.6674,
           0.9150, 0.7840],
          [0.1847, 0.5535, 0.6011, 0.5433, 0.1122, 0.1387, 0.7824, 0.3964,
           0.5836, 0.0032],
          [0.3672, 0.0614, 0.6729, 0.5840, 0.0016, 0.3817, 0.7127, 0.0014,
           0.1786, 0.9084],
          [0.8757, 0.9873, 0.1219, 0.6754, 0.9230, 0.8386, 0.8947, 0.1473,
           0.8560, 0.3940]],

         [[0.3179, 0.3308, 0.9132, 0.1591, 0.3336, 0.0413, 0.8887, 0.3742,
           0.0330, 0.3727],
          [0.1295, 0.3205, 0.5178, 0.5268, 0.7565, 0.5705, 0.2365, 0.8082,
           0.8561, 0.6492],
          [0.1867, 0.9840, 0.7758, 0.6262, 0.2417, 0.2887, 0.2965, 0.4177,
           0.7826, 0.4871],
          [0.6364, 0.6007, 0.5918, 0.4352, 0.0487, 0.6366, 0.1348, 0.8677,
           0.2871, 0.1356]],

         [[0.4954, 0.3594, 0.3975, 0.0518, 0.9698, 0.3253, 0.7577, 0.4650,
           0.4150, 0.4991],
          [0.9952, 0.9912, 0.1387, 0.1541, 0.1930, 0.5015, 0.4293, 0.

Here `sents_emb_keys[i, j, k]` means the attention key vector in the i-th sentence, j-th word, k-th head.

In [10]:
# key vector for 0-th, 3-rd word, 2-nd head
sents_emb_keys[0, 3, 2] 

tensor([0.9587, 0.1933, 0.1780, 0.1439, 0.7586, 0.4737, 0.7456, 0.0403, 0.6641,
        0.2561])

### Caveat: always keep in mind the meaning of shapes of tensors

It is important to always have a record of the meaning of the shape, otherwise one would quickly forget what each tensor dimension means.

In [11]:
sents_emb_keys.size()

torch.Size([2, 5, 4, 10])

Question: what does [2, 5, 4, 10] mean?

Answer: it means [batch_size = 2, sentence_length = 5, number_of_heads = 4, hidden_size = 10], where `number_of_heads` means the number of heads used in the attention mechanism.

## Basic Tensor Operation

### Indexing

In [12]:
# first dimension
sents_emb_keys[0]   # 0-th sentence, all words, all heads, all hidden

tensor([[[0.9654, 0.7689, 0.3381, 0.4762, 0.1286, 0.4259, 0.8224, 0.8908,
          0.8797, 0.9354],
         [0.4685, 0.8491, 0.3870, 0.5836, 0.8562, 0.8005, 0.2111, 0.8136,
          0.9285, 0.4225],
         [0.4361, 0.5648, 0.7100, 0.8261, 0.4496, 0.5599, 0.0967, 0.7952,
          0.2972, 0.5106],
         [0.7805, 0.6896, 0.7740, 0.6492, 0.6878, 0.2625, 0.8063, 0.2552,
          0.0272, 0.8594]],

        [[0.0183, 0.0201, 0.8848, 0.9264, 0.9478, 0.4991, 0.8272, 0.8531,
          0.3745, 0.8296],
         [0.7327, 0.9333, 0.6290, 0.9319, 0.8403, 0.9386, 0.4595, 0.3015,
          0.1834, 0.2354],
         [0.7939, 0.3729, 0.8950, 0.7705, 0.0322, 0.3933, 0.2047, 0.7017,
          0.0232, 0.8769],
         [0.0367, 0.6823, 0.2287, 0.7943, 0.6674, 0.3734, 0.3077, 0.3839,
          0.0096, 0.2811]],

        [[0.2449, 0.1406, 0.5033, 0.6830, 0.7837, 0.7431, 0.5814, 0.4511,
          0.8850, 0.3344],
         [0.2806, 0.9957, 0.0650, 0.5392, 0.2107, 0.5859, 0.4429, 0.3414,
          0.9

In [13]:
sents_emb_keys[0].size()

torch.Size([5, 4, 10])

Q: what does [5, 4, 10] mean?

A: for the 0-th sentence, it has [length = 5, number_of_heads = 4, hidden_size = 10]

In other words, fixing the index of the first dimension of a four-dimensional tensor will result in a three-dimensional tensor.

In [14]:
sents_emb_keys[:, 1, :, :]  # All sentence, 1-st words, all heads, all hidden

tensor([[[0.0183, 0.0201, 0.8848, 0.9264, 0.9478, 0.4991, 0.8272, 0.8531,
          0.3745, 0.8296],
         [0.7327, 0.9333, 0.6290, 0.9319, 0.8403, 0.9386, 0.4595, 0.3015,
          0.1834, 0.2354],
         [0.7939, 0.3729, 0.8950, 0.7705, 0.0322, 0.3933, 0.2047, 0.7017,
          0.0232, 0.8769],
         [0.0367, 0.6823, 0.2287, 0.7943, 0.6674, 0.3734, 0.3077, 0.3839,
          0.0096, 0.2811]],

        [[0.3183, 0.5212, 0.0634, 0.0529, 0.7391, 0.0951, 0.1299, 0.0450,
          0.9903, 0.3069],
         [0.1189, 0.5413, 0.8235, 0.2123, 0.8520, 0.0638, 0.1025, 0.1105,
          0.3888, 0.0779],
         [0.3216, 0.0270, 0.0798, 0.2138, 0.7519, 0.9796, 0.6267, 0.3120,
          0.2358, 0.9861],
         [0.7409, 0.0600, 0.8958, 0.3956, 0.2583, 0.9841, 0.7147, 0.5213,
          0.7362, 0.2340]]])

In [15]:
sents_emb_keys[:, 1, :, :].size()

torch.Size([2, 4, 10])

Q: what does [2, 4, 10] mean?

A: when fixing the 1-st word, we have [batch_size = 2, number_of_heads = 4, hidden_size = 10].

Similarlly we have:

In [16]:
sents_emb_keys[0, :, :, :] # 0-st sentence, all words, all heads, all hidden
sents_emb_keys[:, 0, :, :] # all sentence, 0-th word, all heads, all hidden
sents_emb_keys[:, :, 0, :] # all sentence, all words, 0-th head, all hidden
sents_emb_keys[:, :, :, 0] # all sentence, all words, all heads, 0-th hidden

tensor([[[0.9654, 0.4685, 0.4361, 0.7805],
         [0.0183, 0.7327, 0.7939, 0.0367],
         [0.2449, 0.2806, 0.0512, 0.0748],
         [0.1542, 0.8968, 0.9587, 0.2312],
         [0.1394, 0.9851, 0.9899, 0.1419]],

        [[0.3742, 0.0244, 0.0438, 0.7269],
         [0.3183, 0.1189, 0.3216, 0.7409],
         [0.2885, 0.3024, 0.8328, 0.0341],
         [0.7774, 0.7344, 0.6158, 0.6605],
         [0.1123, 0.1531, 0.3796, 0.2223]]])

In [18]:
# Q: interpret what the following shape means: 
# A: --- YOUR ANSWER HERE ----  # sents_emb_keys = torch.rand([2, 5, 4, 10]) 
print(sents_emb_keys[0, :, :, :].size())
print(sents_emb_keys[:, 0, :, :].size())
print(sents_emb_keys[:, :, 0, :].size())
print(sents_emb_keys[:, :, :, 0].size())

torch.Size([5, 4, 10])
torch.Size([2, 4, 10])
torch.Size([2, 5, 10])
torch.Size([2, 5, 4])


The index can be further fixed at multiple dimensions, for example:

In [19]:
print(sents_emb_keys[0, 1, :, :]) # 0-th sentence, 1-st word, all heads, all hidden
print(sents_emb_keys[0, 1, 2, :]) # 0-th sentence, 1-st word, 2-nd head, all hidden
print(sents_emb_keys[:, 1, 2, :]) # all sentences, 1-st word, 2-nd head, all hidden

tensor([[0.3179, 0.3308, 0.9132, 0.1591, 0.3336, 0.0413, 0.8887, 0.3742, 0.0330,
         0.3727],
        [0.1295, 0.3205, 0.5178, 0.5268, 0.7565, 0.5705, 0.2365, 0.8082, 0.8561,
         0.6492],
        [0.1867, 0.9840, 0.7758, 0.6262, 0.2417, 0.2887, 0.2965, 0.4177, 0.7826,
         0.4871],
        [0.6364, 0.6007, 0.5918, 0.4352, 0.0487, 0.6366, 0.1348, 0.8677, 0.2871,
         0.1356]])
tensor([0.1867, 0.9840, 0.7758, 0.6262, 0.2417, 0.2887, 0.2965, 0.4177, 0.7826,
        0.4871])
tensor([[0.1867, 0.9840, 0.7758, 0.6262, 0.2417, 0.2887, 0.2965, 0.4177, 0.7826,
         0.4871],
        [0.5468, 0.6296, 0.0364, 0.3594, 0.4154, 0.6182, 0.7581, 0.9338, 0.5036,
         0.1573]])


Looking at the shape of the resulting tensors:

In [21]:
sents_emb_keys[:, 1, 2, :].size()

torch.Size([2, 10])

[2, 10] means [batch_size = 2, hidden_size = 10] when fixing the 1-st word and the 2-nd head.

In [22]:
# Q: interpret what the following shape means: 
# A: ---- YOUR ANSWER HERE ---
print(sents_emb_keys[0, 1, :, :].size())
print(sents_emb_keys[0, 1, 2, :].size())

torch.Size([4, 10])
torch.Size([10])


### Slicing

Slicing takes a range of an index within any dimension:

In [23]:
sents_emb_keys[:, 1:4, :, :] # all sentences, 1-3 words (right boundary 4 is not inclusive), all heads, all hidden
sents_emb_keys[:, :4, :, :]  # all sentences, 0-3 words (when left boundary is unspecified, the default is 0), all heads, all hidden
sents_emb_keys[:, 1:, :, :]  # all sentences, 1-last words (when right boundary is unspecified, the default is the last token), all heads, all hidden
sents_emb_keys[:, :, 2:4:, :] # all sentences, all words, 2-3 heads, all hidden
sents_emb_keys[:, :, :, 1:9] # all sentences, all words, all heads, 1-8 hidden

tensor([[[[0.8729, 0.9025, 0.6452, 0.3510, 0.8613, 0.7899, 0.6674, 0.9150],
          [0.5535, 0.6011, 0.5433, 0.1122, 0.1387, 0.7824, 0.3964, 0.5836],
          [0.0614, 0.6729, 0.5840, 0.0016, 0.3817, 0.7127, 0.0014, 0.1786],
          [0.9873, 0.1219, 0.6754, 0.9230, 0.8386, 0.8947, 0.1473, 0.8560]],

         [[0.3308, 0.9132, 0.1591, 0.3336, 0.0413, 0.8887, 0.3742, 0.0330],
          [0.3205, 0.5178, 0.5268, 0.7565, 0.5705, 0.2365, 0.8082, 0.8561],
          [0.9840, 0.7758, 0.6262, 0.2417, 0.2887, 0.2965, 0.4177, 0.7826],
          [0.6007, 0.5918, 0.4352, 0.0487, 0.6366, 0.1348, 0.8677, 0.2871]],

         [[0.3594, 0.3975, 0.0518, 0.9698, 0.3253, 0.7577, 0.4650, 0.4150],
          [0.9912, 0.1387, 0.1541, 0.1930, 0.5015, 0.4293, 0.3002, 0.4657],
          [0.6919, 0.2448, 0.5146, 0.3169, 0.1226, 0.3471, 0.1492, 0.4953],
          [0.2616, 0.2614, 0.6155, 0.7317, 0.2351, 0.5597, 0.1290, 0.3548]],

         [[0.6291, 0.6374, 0.2383, 0.4704, 0.6025, 0.7852, 0.9046, 0.4481],
      

You can also set a stepsize for slicing:

In [24]:
sents_emb_keys[:, 0::2, :, :] # all sentence, 0, 2, 4 words

tensor([[[[0.8334, 0.8729, 0.9025, 0.6452, 0.3510, 0.8613, 0.7899, 0.6674,
           0.9150, 0.7840],
          [0.1847, 0.5535, 0.6011, 0.5433, 0.1122, 0.1387, 0.7824, 0.3964,
           0.5836, 0.0032],
          [0.3672, 0.0614, 0.6729, 0.5840, 0.0016, 0.3817, 0.7127, 0.0014,
           0.1786, 0.9084],
          [0.8757, 0.9873, 0.1219, 0.6754, 0.9230, 0.8386, 0.8947, 0.1473,
           0.8560, 0.3940]],

         [[0.4954, 0.3594, 0.3975, 0.0518, 0.9698, 0.3253, 0.7577, 0.4650,
           0.4150, 0.4991],
          [0.9952, 0.9912, 0.1387, 0.1541, 0.1930, 0.5015, 0.4293, 0.3002,
           0.4657, 0.6610],
          [0.5574, 0.6919, 0.2448, 0.5146, 0.3169, 0.1226, 0.3471, 0.1492,
           0.4953, 0.1999],
          [0.6792, 0.2616, 0.2614, 0.6155, 0.7317, 0.2351, 0.5597, 0.1290,
           0.3548, 0.1472]],

         [[0.6946, 0.7394, 0.8725, 0.4858, 0.1086, 0.7844, 0.3104, 0.5699,
           0.0870, 0.3478],
          [0.6840, 0.8955, 0.0373, 0.8702, 0.9629, 0.6660, 0.2640, 0.

In [25]:
sents_emb_keys[:, 1::2, :, :] # # all sentence, 1, 3 words

tensor([[[[0.3179, 0.3308, 0.9132, 0.1591, 0.3336, 0.0413, 0.8887, 0.3742,
           0.0330, 0.3727],
          [0.1295, 0.3205, 0.5178, 0.5268, 0.7565, 0.5705, 0.2365, 0.8082,
           0.8561, 0.6492],
          [0.1867, 0.9840, 0.7758, 0.6262, 0.2417, 0.2887, 0.2965, 0.4177,
           0.7826, 0.4871],
          [0.6364, 0.6007, 0.5918, 0.4352, 0.0487, 0.6366, 0.1348, 0.8677,
           0.2871, 0.1356]],

         [[0.7748, 0.6291, 0.6374, 0.2383, 0.4704, 0.6025, 0.7852, 0.9046,
           0.4481, 0.4605],
          [0.3760, 0.9614, 0.7614, 0.9325, 0.5496, 0.3769, 0.4365, 0.7999,
           0.4480, 0.7807],
          [0.8757, 0.0044, 0.5574, 0.3041, 0.5696, 0.5299, 0.2368, 0.4774,
           0.0051, 0.7699],
          [0.7108, 0.2217, 0.4519, 0.1291, 0.3758, 0.5096, 0.4929, 0.7431,
           0.8968, 0.4357]]],


        [[[0.2905, 0.4046, 0.1233, 0.3721, 0.8163, 0.5609, 0.9449, 0.3421,
           0.6740, 0.0334],
          [0.5053, 0.4452, 0.7289, 0.3012, 0.6445, 0.5195, 0.0187, 

In [26]:
sents_emb_keys[:, 0:4:2, :, :] # # all sentence, 0, 2 words

tensor([[[[0.8334, 0.8729, 0.9025, 0.6452, 0.3510, 0.8613, 0.7899, 0.6674,
           0.9150, 0.7840],
          [0.1847, 0.5535, 0.6011, 0.5433, 0.1122, 0.1387, 0.7824, 0.3964,
           0.5836, 0.0032],
          [0.3672, 0.0614, 0.6729, 0.5840, 0.0016, 0.3817, 0.7127, 0.0014,
           0.1786, 0.9084],
          [0.8757, 0.9873, 0.1219, 0.6754, 0.9230, 0.8386, 0.8947, 0.1473,
           0.8560, 0.3940]],

         [[0.4954, 0.3594, 0.3975, 0.0518, 0.9698, 0.3253, 0.7577, 0.4650,
           0.4150, 0.4991],
          [0.9952, 0.9912, 0.1387, 0.1541, 0.1930, 0.5015, 0.4293, 0.3002,
           0.4657, 0.6610],
          [0.5574, 0.6919, 0.2448, 0.5146, 0.3169, 0.1226, 0.3471, 0.1492,
           0.4953, 0.1999],
          [0.6792, 0.2616, 0.2614, 0.6155, 0.7317, 0.2351, 0.5597, 0.1290,
           0.3548, 0.1472]]],


        [[[0.2559, 0.2559, 0.9162, 0.7375, 0.6582, 0.6377, 0.5439, 0.2733,
           0.1401, 0.1108],
          [0.3211, 0.0384, 0.0264, 0.1529, 0.9963, 0.0339, 0.8921, 

As you can see, the slicing syntax is `some_tensor[start_index : end_index : step_size]`.

In [27]:
# Q: interpret what the following shape means:
# A: ---- YOUR ANSWER HERE ----
print(sents_emb_keys[:, 1:4, :, :].size())
print(sents_emb_keys[:, :, 3:, :].size())

torch.Size([2, 3, 4, 10])
torch.Size([2, 5, 1, 10])


### Concatenation

To concatenate two tensors within a specified dimension:

In [28]:
sent_1 = torch.tensor([[1, 2, 3]]) # size = [1, 3]
sent_2 = torch.tensor([[4, 5, 6]]) # size = [1, 3]

In [34]:
# concatenate two sentences into a batch
sents = torch.cat([sent_1, sent_2], dim=0)
print(sents)
print(sents.size())

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])


In [35]:
# concat the second sentence to the end of the first sentence
sents1 = torch.cat([sent_1, sent_2], dim=1)
print(sents1)
print(sents1.size())

tensor([[1, 2, 3, 4, 5, 6]])
torch.Size([1, 6])


### Reshaping

Usually reshaping has two effects: (a) spliting tensors or (b) concatenating tensors.

In [36]:
# concatenating tensors
print(sents)
print(sents.size())
sents_cat = sents.view(6)
print(sents_cat)
print(sents_cat.size()) # pay attention how the shape of the tensor changes from [2, 3] -> [6]

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])
tensor([1, 2, 3, 4, 5, 6])
torch.Size([6])


In [37]:
# spliting tensors
sents_split_1 = sents_cat.view(2, 3)
print(sents_split_1)
print(sents_split_1.size())
sents_split_2 = sents_cat.view(3, 2)
print(sents_split_2)
print(sents_split_2.size())

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
torch.Size([3, 2])


### Transposing

Transpose the two dimensions of a tensor:

In [38]:
print(sents) # shape = [batch, length]
sents_transposed = sents.transpose(0, 1)
print(sents_transposed) # shape = [length, batch]

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])


### Caveat: pay close attention to the difference between transposing and reshaping

Generally, reshaping does not change the order of the items within the tensor, while tranposing changes the order.

In [39]:
# view does not change order
print(sents.view(6))
print(sents.view(2, 3))
print(sents.view(3, 2))

# transposing changes the order
print(sents.transpose(0, 1))
print(sents.transpose(0, 1).reshape(6))

tensor([1, 2, 3, 4, 5, 6])
tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([1, 4, 2, 5, 3, 6])


## Basic Tensor Computations

### Tensor add scalar

In [40]:
# addition, differences, multiplication, division
sent_emb = torch.rand(3, 5)    # [length = 5, hidden = 10] 
                                # we assume we work with one sentence embedding where there are 3 words and 
                                # each word is associated with an embedding vector with length 5
print(sent_emb)

# the following operations are applied to every entry within the tensor
print(sent_emb + 1)
print(sent_emb - 1)
print(sent_emb * 2)
print(sent_emb / 2)

tensor([[0.3594, 0.2411, 0.6584, 0.6098, 0.4540],
        [0.4520, 0.4716, 0.0068, 0.6186, 0.3309],
        [0.8766, 0.3103, 0.1715, 0.1097, 0.3585]])
tensor([[1.3594, 1.2411, 1.6584, 1.6098, 1.4540],
        [1.4520, 1.4716, 1.0068, 1.6186, 1.3309],
        [1.8766, 1.3103, 1.1715, 1.1097, 1.3585]])
tensor([[-0.6406, -0.7589, -0.3416, -0.3902, -0.5460],
        [-0.5480, -0.5284, -0.9932, -0.3814, -0.6691],
        [-0.1234, -0.6897, -0.8285, -0.8903, -0.6415]])
tensor([[0.7188, 0.4822, 1.3168, 1.2195, 0.9080],
        [0.9039, 0.9433, 0.0136, 1.2373, 0.6618],
        [1.7532, 0.6205, 0.3429, 0.2194, 0.7170]])
tensor([[0.1797, 0.1205, 0.3292, 0.3049, 0.2270],
        [0.2260, 0.2358, 0.0034, 0.3093, 0.1655],
        [0.4383, 0.1551, 0.0857, 0.0549, 0.1793]])


### Vector dot product

In [41]:
# vector dot-product
word_1 = torch.rand(5)
word_2 = torch.rand(5)

prod = (word_1 * word_2).sum() # element-wise multiplication then sum 

In [43]:
word_1 = torch.tensor([1, 2, 3])
word_2 = torch.tensor([1, 1, 1])
prod = (word_1 * word_2).sum()
prod

tensor(6)

### Matrix multiplication

In [44]:
# matrix mulplication
M1 = torch.rand(2, 5)
M2 = torch.rand(5, 10)

prod = torch.matmul(M1, M2)
print(prod.size()) # shape changes: [2, 5] * [5, 10] -> [2, 10]

torch.Size([2, 10])


## Alignment and broadcasting

### Rule 1: if the same shape, then element-wise computation

In [45]:
# adding two words, each word is a vector
word_1 = torch.rand(5) # hidden_size = 5
word_2 = torch.rand(5)
result = word_1 + word_2 # result[i] = word_1[i] + word_2[i] 

In [46]:
# adding two sentences, each sentence is a sequence of word vectors
sent_1 = torch.rand(3, 5) # sent_length = 3, hidden_size = 5
sent_2 = torch.rand(3, 5)
result = sent_1 + sent_2 # result[i, j] = sent_1[i, j] + sent_2[i, j]

### Rule 2: if different shape, first align, then element-wise computation

In [47]:
# Example 1. adding a single word to every word within a sentence
word = torch.tensor([1, 2, 3])              # [hidden = 3]
sent = torch.tensor([[0, 0, 0], 
                     [1, 1, 1]]) # [sent_length = 2, hidden = 3]

result = word.view(1, 3) + sent # underlying process: 
                                # step 1, alignment
                                # word.size() = [1, 3]
                                #                |  |   alignment
                                # sent.size() = [2, 3]
                                #
                                # step 2, repeat where dimension size is 1
                                # word_repeat = word.repeat([2, 1]) -- 0th dim repeat twice, first dim repeat 1 time (= no repeat)
                                #
                                # step 3, align again
                                # align again
                                # word_repeat.size() = [2, 3]
                                #                       |  |   alignmeng, note that the 0th dim of word is repeated
                                # sent.size()        = [2, 3]
                                #
                                # step 4, element-wise addition. 
                                # result = word_repeat + sent
print(result)  # this is basically broadcasing the given word to each word in the given sentence

tensor([[1, 2, 3],
        [2, 3, 4]])


In [48]:
# Example 2. adding two words to two sentences, respectively
words = torch.tensor([[1, 2, 3], [4, 5, 6]])                      # batch_size = 2, hidden_size = 3
sents = torch.tensor([
                        [[0, 0, 0], [1, 1, 1], [2, 2, 2]], 
                        [[3, 3, 3], [4, 4, 4], [5, 5, 5]],        # batch_size = 2, length = 2, hidden_size = 3
                    ])

# we aim to add words[0] to sents[0] and words[1] to sents[1]
result = words.view(2, 1, 3) + sents    # underlying process: 
                                        # 
                                        # step 1, alignment
                                        # words.size() = [2, 1, 3]   two words, each has hidden dim = 3
                                        #                 |  |  |    alignment
                                        # sents.size() = [2, 2, 3]   two sentences, each has two words, each word has hidden dim = 3
                                        # 
                                        # step 2, repeat where dimension size is 1 for broadcasting
                                        # repeat words along 1st dimension
                                        # words_repeat = words.repeat([1, 2, 1]) -- 0th dim repeat 1 time, 1st dim repeat 2 times, 3rd dim repeat 1 time
                                        # 
                                        # step 3, align again
                                        # words_repeat.size() = [2, 2, 3]
                                        #                        |  |  |   alignment, note that the 1st dim of words is repeated
                                        # sents.size()        = [2, 2, 3]
                                        #
                                        # step 4, element-wise addition
                                        # result = words_repeat + sents
print(result)
print(result.size())

tensor([[[ 1,  2,  3],
         [ 2,  3,  4],
         [ 3,  4,  5]],

        [[ 7,  8,  9],
         [ 8,  9, 10],
         [ 9, 10, 11]]])
torch.Size([2, 3, 3])


In [49]:
# Example 3, outer product
# the outer product of two vectors is a matrix
v1 = torch.tensor([1, 2, 3])
v2 = torch.tensor([4, 5, 6])

outer_prod = v1.view(3, 1) * v2.view(1, 3)  # underlying process: 
                                            # 
                                            # step 1, alignment
                                            # v1.size() = [3, 1]  
                                            #              |  |  alignment
                                            # v2.size() = [1, 3] 
                                            # 
                                            # step 2, repeat where dimension size is 1
                                            # repeat v1 along 1st dimension
                                            # v1_repeat = v1.repeat([1, 3]) -- 0th dim repeat 1 time, 1st repeat 3 times
                                            # repeat v2 along 0th dimension
                                            # v2_repeat = v2.repeat([3, 1]) -- 0th dim repeat 3 times, 1st repeat 1 time
                                            #
                                            # step 3, align again and element-wise computation
                                            # v1_repeat.size() = [3, 3]
                                            #                     |  | 
                                            # v2_repeat.size() = [3, 3]
                                            #
                                            # step 4, elementwise multiplication
                                            # outer_prod = v1_repeat * v2_repeat

print(outer_prod)

tensor([[ 4,  5,  6],
        [ 8, 10, 12],
        [12, 15, 18]])


In [53]:
# Pay attention to the shape of the tensor, again
v1 = torch.tensor([1, 2, 3])
v2 = torch.tensor([4, 5, 6])

# Q: what is the differences between the following two operations: 
# A: ---- YOUR ANSWER HERE ----
result_1 = v1.view(3, 1) * v2.view(1, 3)
result_2 = v1.view(1, 3) * v2.view(3, 1)
print(result_1)
print(result_2)

tensor([[ 4,  5,  6],
        [ 8, 10, 12],
        [12, 15, 18]])
tensor([[ 4,  8, 12],
        [ 5, 10, 15],
        [ 6, 12, 18]])


# Section 2. Batchified Tensor Computation

In this section, we will study how to compute with tensors whose first dimension is the batch size.

## Batch matrix multiplication

In [54]:
# linear transform of the word embeddings of two sentences
sents = torch.rand(2, 5, 10) # [batch_size = 2, length = 2, hidden_size= 10]
weight = torch.rand(10, 20)  # we transform each word from a length-10 vector to a length-20 vector

sents_transform = torch.matmul(sents, weight)   # underlying process:
                                                # sents.size() = [2,  5, 10]
                                                #                 |       |         alignment. 
                                                # view weight as [1,     10, 20]    repeat where dimension is 1
                                                # matrix multiplication happens at the final two dimensions, [2, 5, 10] x [10, 20] -> [2, 5, 20]
print(sents_transform.size())

torch.Size([2, 5, 20])


## Retrieving embedding vectors from an embedding matrix

In [55]:
# in deep learning practice, usually we start with sequences of word index
sent = torch.tensor([[10, 25, 59, 77, 88],  # suppose [10, 25, 59, 77, 88] means = ['Oh', I', 'really', 'like', 'cats']
                     [16, 29, 40, 56, 3]]   # suppose [16, 29, 40, 56, 3] means = ['Jack', 'does', 'not', 'have', 'dogs']
                    )
print(sent.size())

# neural networks requires words being represented as embedding vectors, not index. So we store an embedding matrix for every word
vocab_size = 100
hidden_size = 10
embedding_matrix = torch.rand([vocab_size, hidden_size])

# converting index to embeddings consists of two steps:
# 1. convert index representation to one-hot representation
sent_one_hot = F.one_hot(sent, vocab_size).float()
# 2. batch matrix multiplication between the one hot representation and the embedding matrix
#    NOTE: when multiply a one-hot vector to a matrix, the one-hot vector essentially retrieves the corresponding row vector from the matrix
sent_emb = torch.matmul(sent_one_hot, embedding_matrix) # underlying process:
                                                        # sent_one_hot.size() =    [2,  5, 100]
                                                        #                           |       |         alignment. 
                                                        # view embedding_matrix as [1,     100, 10]   repeat where dimension is 1
                                                        # matrix multiplication happens at the final two dimensions, [2, 5, 100] x [100, 10] -> [2, 5, 10]
                                                        # In this case, multiplying an one-hot vector to a matrix means to retrieve the corresponding 
                                                        # row vector from the matrix

print(sent_emb.size())

# the following show the embedding vector from word index 10 is retrieved for sent[0, 0]
print(sent_emb[0, 0])
print(embedding_matrix[10])

# similarlly the embedding vector from word index 25 is retrieved for sent[0, 1]
print(sent_emb[0, 1])
print(embedding_matrix[25])

# ---- YOUR TASK ----
# verify how one-hot vector can retrieve the corresponding row vector from the matrix

torch.Size([2, 5])
torch.Size([2, 5, 10])
tensor([0.9432, 0.9382, 0.6470, 0.1222, 0.3370, 0.7655, 0.0559, 0.8464, 0.0360,
        0.0308])
tensor([0.9432, 0.9382, 0.6470, 0.1222, 0.3370, 0.7655, 0.0559, 0.8464, 0.0360,
        0.0308])
tensor([0.3585, 0.0922, 0.4751, 0.5944, 0.1077, 0.4041, 0.0765, 0.9159, 0.2264,
        0.6065])
tensor([0.3585, 0.0922, 0.4751, 0.5944, 0.1077, 0.4041, 0.0765, 0.9159, 0.2264,
        0.6065])


## Linear transform of the word embeddings

In [56]:
# linear transform of the word embeddings of two sentences where each sentence has 4 attention heads
# this computation is performed in multi-head attention in CW2

sents = torch.rand(2, 5, 4, 10) # [batch_size = 2, length = 2, number_of_heads = 4, hidden_size= 10]
weight = torch.rand(10, 20)  # we transform each word from a length-10 vector to a length-20 vector

sents_transform = torch.matmul(sents, weight)   # underlying process:
                                                # sents.size() = [2,  5, 4, 10]
                                                #                 |   |      |         alignment. 
                                                # view weight as [1,  1,    10, 20]    repeat where dimension is 1
                                                # matrix multiplication happens at the final two dimensions, [2, 5, 4, 10] * [10, 20] -> [2, 5, 4, 20]
print(sents_transform.size())

torch.Size([2, 5, 4, 20])


## Similarity: one single vector v.s. a batch of sentence 

In [58]:
# similarity: one single vector v.s. a batch of sentence representations
query = torch.rand(10)
sents = torch.rand(2, 5, 10)

# suppose we would like to compute the dot-product (as a measure of similarity) between the query vector and all words within all sentences
similarity = (query.view(1, 1, 10) * sents)
print(similarity.size())  # similarity[i, j] means the vector dot-product between the query vector and the word j at sentence i

torch.Size([2, 5, 10])


## Similarity: a batch of vectors v.s. a batch of sentence  

In [57]:
# similarity: a batch of vectors v.s. a batch of sentence representations 
query = torch.rand(2, 10) # pay attention to the differences to the previous case
sents = torch.rand(2, 5, 10)

similarity = (query.view(2, 1, 10) * sents).sum(dim=2)
print(similarity.size())  # similarity[i, j] means the vector dot-product between the query[i] and the word j at sentence i

torch.Size([2, 5])


## Similarity: a sentence v.s. a batch of sentence

In [59]:
# similarity: a sentence v.s. a batch of sentence representations
sent_0 = torch.rand(4, 10)
sents = torch.rand(2, 5, 10)

# Q: what is the underlying process? hint: recall the previous align-and-broadcast
# A: ---- YOUR ANSWER HERE ----
similarity = (sent_0.view(1, 4, 1, 10) * sents.view(2, 1, 5, 10)).sum(dim=3) 
print(similarity.size()) # similarity[i, j, k] means the vector dot-product between sent_0[j] (j-th word) and sents[i, k] (i-th sent, k-th word)

similarity = (sent_0.view(1, 1, 4, 10) * sents.view(2, 5, 1, 10)).sum(dim=3)
print(similarity.size()) # similarity[i, j, k] means the vector dot-product between sent_0[k] (k-th word) and sents[i, j] (i-th sent, j-th word)

# NOTE: pay attention two the differences of the shapes

torch.Size([2, 4, 5])
torch.Size([2, 5, 4])


## Similarity: two batches of sentences

In [60]:
# similarity: two batches of sentences representations
# this is usually happens in machine translation where we have a batch of sentences of source language, and another batch of sentences of target language
source_sents = torch.rand(2, 5, 10) # [batch_size = 2, source_sent_length = 5, hidden_size = 10]
target_sents = torch.rand(2, 6, 10) # [batch_size = 2, target_sent_length = 6, hidden_size = 10]

# Q: what is the underlying process? hint: recall the previous align-and-broadcast
# A: ---- YOUR ANSWER HERE ----
similarity = (source_sents.view(2, 5, 1, 10) * target_sents.view(2, 1, 6, 10)).sum(dim = 3)
print(similarity.size()) # similarity[i, j, k] means source_sents[i, j] v.s. target_sents[i, k]

similarity = (source_sents.view(2, 1, 5, 10) * target_sents.view(2, 6, 1, 10)).sum(dim = 3)
print(similarity.size()) # similarity[i, j, k] means source_sents[i, k] v.s. target_sents[i, j]
# NOTE: pay attention two the differences of the shapes

torch.Size([2, 5, 6])
torch.Size([2, 6, 5])


# Section 3. Masked Batch Computation

When putting sentences of difference length into a batch, common practice is to pad them into the same pre-specified maximum length with a special PAD token, then mask out computation involving the PAD token.

## Basic Masking

In [61]:
# a batch of sentences with different length
PAD = 0
sents = torch.tensor([[1, 2, 3, 0, 0], # length = 3, 2 PAD tokens,
                      [3, 4, 0, 0, 0], # length = 2, 3 PAD tokens
                      [5, 6, 7, 8, 9]] # length = 5, no PAD token
                    )

In [62]:
# create a batch of mask vectors
mask = (sents != PAD).float()
print(mask)

tensor([[1., 1., 1., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1.]])


## Masked Batchified Computation

Now we repeat the computation of the previous section. But this time, we mask out the computation involving the PAD tokens.

## Similarity: one single vector vs. a batch of sentence + mask

In [63]:
# one single vector v.s. a batch of sentence representations + a batch of mask
query = torch.rand(10)
sents = torch.rand(3, 5, 10)

# suppose we would like to compute the dot-product (as a measure of similarity) between the query vector and all words within all sentences
similarity = (query.view(1, 1, 10) * sents * mask.view(3, 5, 1)).sum(dim=2) # query.size = [1, 1, 10]
                                                                            # sents.size = [3, 5, 10]
                                                                            # mask.size  = [3, 5, 1]
                                                                            # recall the align-then-broadcast introduced in the previous section
print(similarity)
print(similarity.size())  # similarity[i, j] means the vector dot-product between the query vector and the word j at sentence i
                          # similarity scores for PAD tokens are masked to be 0

tensor([[2.5105, 2.2764, 2.8527, 0.0000, 0.0000],
        [2.4761, 1.7679, 0.0000, 0.0000, 0.0000],
        [4.0252, 2.5680, 2.1620, 2.9131, 4.3537]])
torch.Size([3, 5])


## Similarity: a sentence vs. a batch of sentence + mask

In [64]:
# similarity: a sentence v.s. a batch of sentence representations
sent_0 = torch.rand(4, 10)
sent_0_mask = torch.tensor([1, 1, 1, 0]) # we assume the length of sent_0 is 3

sents = torch.rand(3, 5, 10)

# Q: what is the underlying process? hint: recall the previous align-and-broadcast
# A: ---- YOUR ANSWER HERE ----
#
# hint: first mask out sent_0
#       sent_0.size        = [1, 4, 1, 10]
#       sent_0_mask.size   = [1, 4, 1, 1]
#       sent_0_masked.size = [1, 4, 1, 10]
#
#       then mask out sents
#       sents.size         = [3, 1, 5, 10]
#       mask.size          = [3, 1, 5, 1 ]
#       sents_masked.size  = [3, 1, 5, 10]
#
#       finally computed similarity score for the two masked tensors
#       sent_0_masked.size = [1, 4, 1, 10]
#       sents_masked.size  = [3, 1, 5, 10]
#       similarity.size    = [3, 4, 5]

similarity = (sent_0.view(1, 4, 1, 10) * sent_0_mask.view(1, 4, 1, 1) * sents.view(3, 1, 5, 10) * mask.view(3, 1, 5, 1)).sum(dim=3) 

print(similarity)
print(similarity.size()) # similarity[i, j, k] means the vector dot-product between sent_0[j] (j-th word) and sents[i, k] (i-th sent, k-th word)
                         # similarity scores for PAD tokens are masked to be 0

tensor([[[1.3298, 1.4957, 1.2587, 0.0000, 0.0000],
         [2.2465, 2.6633, 2.3970, 0.0000, 0.0000],
         [1.5583, 1.8176, 1.7344, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[1.3758, 1.7666, 0.0000, 0.0000, 0.0000],
         [2.2330, 2.6583, 0.0000, 0.0000, 0.0000],
         [1.5869, 1.9688, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[2.1724, 1.1920, 1.0303, 1.3125, 1.0763],
         [3.1026, 1.9577, 2.0134, 1.7060, 2.1650],
         [2.9045, 1.9501, 1.5935, 1.9299, 1.8313],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]])
torch.Size([3, 4, 5])


## Similarity: two batches of masked sentences

In [151]:
# similarity: two batches of sentences representations
# this is usually happens in machine translation where we have a batch of sentences of source language, and another batch of sentences of target language
# here additionaly, we assume each sentence has its own mask 
source_sents = torch.rand(2, 5, 10) # [batch_size = 2, source_sent_length = 5, hidden_size = 10]
source_mask  = torch.tensor([[1, 1, 1, 1, 0], # length = 4
                             [1, 1, 1, 1, 1]] # length = 5
                           )
target_sents = torch.rand(2, 6, 10) # [batch_size = 2, target_sent_length = 6, hidden_size = 10]
target_mask  = torch.tensor([[1, 1, 1, 0, 0, 0], # length = 3
                             [1, 1, 1, 1, 1, 1]] # length = 6
                           )

# Q: what is the underlying process? hint: recall the previous align-and-broadcast-and-mask
# A: ---- YOUR ANSWER HERE ----
similarity = (source_sents.view(2, 5, 1, 10) * source_mask.view(2, 5, 1, 1) * target_sents.view(2, 1, 6, 10) * target_mask.view(2, 1, 6, 1)).sum(dim = 3)
print(similarity)
print(similarity.size()) # similarity[i, j, k] means source_sents[i, j] v.s. target_sents[i, k]

tensor([[[2.5672, 2.6088, 2.5834, 0.0000, 0.0000, 0.0000],
         [2.0834, 1.5814, 2.3227, 0.0000, 0.0000, 0.0000],
         [2.5505, 2.8746, 2.9579, 0.0000, 0.0000, 0.0000],
         [2.6716, 2.0214, 2.7910, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[2.0494, 2.5466, 2.8356, 2.0703, 2.9785, 3.8640],
         [0.9491, 1.5451, 1.6202, 1.1407, 1.5279, 1.9398],
         [3.0602, 2.4551, 3.5492, 2.8821, 3.5459, 4.9329],
         [2.5598, 2.3238, 2.8777, 2.4559, 2.8768, 4.1201],
         [2.0323, 1.8204, 2.0931, 2.0048, 1.5199, 2.5985]]])
torch.Size([2, 5, 6])


In [67]:
mat = torch.tensor([[1, 2, 3, 4, 5, 6],
                    [7, 8, 9, 10, 11, 12]])
mat.view(3, 1, 4, 1)

tensor([[[[ 1],
          [ 2],
          [ 3],
          [ 4]]],


        [[[ 5],
          [ 6],
          [ 7],
          [ 8]]],


        [[[ 9],
          [10],
          [11],
          [12]]]])

# Section 4. Final Application: Negative Log Likelihood

Finally, we compute the per-word negative log likelihood of a neural language model that puts everything that we have learned together.

The per-word NLL for a sentence $x_1, ..., x_T$ is:

$$
- \frac{1}{T} \sum_t \log p(x_t | x_{1:t-1})
$$

which is the average of word likelihood over sentence length.

In [68]:
# we start with the final layer of a neural language model which output a tensor named logits (logits means the vector before putting into softmax):
logits = torch.rand(2, 5, 100)  # [batch_size = 2, max_sent_len = 5, vocab_size = 100]
                                # in practice this should be the output of the final layer of a neural language model. Here we just simulate it as random tensor
vocab_size = 100 # in practice the vocabulary size is usually >10K, here for demonstration we simplify it to 100

# putting the logits into a softmax function gives the probability of each word within the vocabulary
probs = F.softmax(logits, dim=2)

# negative log likelihood requires the log of probability
log_probs = F.log_softmax(logits, dim=2)

# a language model aims to maximize the probability of each word in every sentence, but the PAD token should be excluded
target_sent = torch.tensor([[10, 25, 59, 77, 0],   # length = 4, suppose [10, 25, 59, 77, 0] means = ['I', 'really', 'like', 'cats', 'PAD']
                            [16, 29, 40, 56, 3]]   # length = 5, suppose [16, 29, 40, 56, 3] means = ['Jack', 'does', 'not', 'have', 'dogs']
                          )

# transform the index representation to the one-hot representation
target_one_hot = F.one_hot(target_sent, vocab_size)

# create mask for the target sentence
mask = (target_sent != 0).float()

# the following computation computes the negative log likelihood
# ---- YOUR TASK ----
# 1. annotate the shape of every tensor in the following computation
# 2. explain every computation step
negative_log_likelihood = (((-target_one_hot * log_probs).sum(dim=2) * mask).sum(dim=1) / mask.sum(dim=1)).mean()

# Further Reading

Up to this stage, you have already suffered from endless annotation of shape of tensor. Is there a way that one can write code which is self-explanatory, rather than requires shape annotation at every single line? 

The answer is yes, and the solution is [einsum](https://rockt.github.io/2018/04/30/einsum) and [einops](https://einops.rocks/). You are encouraged to find them out yourself.